In [1]:
%load_ext autoreload
%autoreload 2
import pickle
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import normalize
from os.path import join
from scipy.stats import entropy
#from functions.dataset import Dataset


### PREPARACION DE LOS DATOS

In [2]:
data = pd.read_csv("C:/Users/dan12/Downloads/churn_uplift_anonymized.csv") # reemplazar por ruta de su pc

In [3]:

from sklearn.model_selection import train_test_split
import numpy as np

class Dataset:
    def __init__(self, **arrays):
        self.__dict__.update(arrays)
    
    def __getitem__(self, key):
        res = Dataset()
        res.__dict__.update(self.__dict__)
        for name in res.__dict__:
            res.__dict__[name] = res.__dict__[name][key].copy()
        return res
        #return Dataset(**{name: value[key] for name, value in self.__dict__.items()})
    
    def train_test_split(self, *args, **kwargs):
        ordered_dict = list(self.__dict__.items())
        res = train_test_split(*list(value for name, value in ordered_dict), *args, **kwargs)
        return (
            Dataset(**{ordered_dict[i][0]: res[i * 2] for i in range(len(ordered_dict))}),
            Dataset(**{ordered_dict[i][0]: res[i * 2 + 1] for i in range(len(ordered_dict))})
        )

In [4]:
exclude_from_X = [
    'y',
    't'
]
columns_X = [c for c in data.columns if c not in exclude_from_X]
dataset = Dataset(
    X = data[columns_X],
    y = data.y == 1,
    t = data.t == 1
)

In [6]:
def replace_less_common(df, k=10, replacement="Other"):
    df = df.copy()
    for col in df.columns:
        if df[col].dtype in ["string", "object", "category"]:
            top_values = df[col].value_counts()[:k]
            df.loc[~df[col].isin(top_values.index), col] = replacement
    return df

In [7]:
# Factor encoding
dataset.X = replace_less_common(dataset.X)
dataset.X = pd.get_dummies(dataset.X).to_numpy().astype("float32")
dataset.X[~np.isfinite(dataset.X)] = 0

# Data normalization
dataset.X = dataset.X[:, dataset.X.std(axis=0) != 0]
normalization = "minmax"
if normalization == "gaussian":
    m = dataset.X.mean(axis=0)
    s = dataset.X.std(axis=0)
    dataset.X = (dataset.X - m) / s
elif normalization == "minmax":
    M = dataset.X.max(axis=0)
    m = dataset.X.min(axis=0)
    dataset.X = (dataset.X - m) / (M - m)

# Target variables
dataset.t = np.array(dataset.t)
dataset.y = np.array(dataset.y)

In [8]:
print(dataset.X)

[[0.01232632 0.94601023 0.6288942  ... 0.         0.         0.        ]
 [0.00565895 0.93717617 0.70662796 ... 0.         0.         0.        ]
 [0.01812158 0.9430231  0.63319874 ... 0.         0.         0.        ]
 ...
 [0.0368864  0.9627086  0.71831447 ... 0.         1.         0.        ]
 [0.03270227 0.9642671  0.7324893  ... 0.         1.         0.        ]
 [0.05129221 0.97584814 0.64182186 ... 0.         1.         0.        ]]


### MODELO DE CHURN

In [9]:
#seed = None

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
X_train, X_test, y_train, y_test = train_test_split(dataset.X, dataset.y, stratify=dataset.y, random_state=42)

# Entrenamos un modelo de Random Forest para predecir churn

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predicción de probabilidad de churn, EVALUAMOS EL MODELO
y_prob = model.predict_proba(X_test)[:, 1]
print("ROC AUC:", roc_auc_score(y_test, y_prob))

ROC AUC: 0.7185008056147251


## CLTV

In [10]:
# Calculamos la probabilidad de churn para todo el dataset
data["churn_prob"] = model.predict_proba(dataset.X)[:, 1]

# Simulamos un CLTV con una función basada en componentes aleatorios
np.random.seed(42)
data["CLTV"] = (
    500 + 
    300 * np.random.rand(len(data)) - 
    200 * data["churn_prob"] + 
    100 * (1 - data["churn_prob"])
)

In [11]:
data

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,FACTOR13,FACTOR14,FACTOR15,FACTOR16,FACTOR17,FACTOR18,y,t,churn_prob,CLTV
0,-3.064007,-1.272259,-4.075844,1.492514,2.328039,-0.330240,-0.770389,0.035541,2.439325,-0.847266,...,V5,V20,V9,V11,V1,V4,0,0,0.00,712.362036
1,-4.574066,-3.541815,1.107371,0.447314,-0.471140,-0.567309,0.195963,0.383654,1.523472,-0.184596,...,V6,V20,V9,V15,V4,V4,0,0,0.01,882.214292
2,-1.751471,-2.039692,-3.788823,0.624226,0.564614,-0.713385,1.003502,1.582819,-1.461687,0.844975,...,V11,V20,V9,V1,V4,V4,0,1,0.02,813.598183
3,-2.030089,-0.235720,-5.711960,1.071841,2.441541,0.758484,-0.534295,1.614289,-1.022909,-1.383025,...,V4,V20,V11,V1,V4,V4,0,1,0.00,779.597545
4,-2.377857,-2.478670,-1.051946,2.812768,2.019267,0.095542,-0.033324,-1.332319,4.316526,-0.537967,...,V7,V20,V2,V1,V2,V2,0,1,0.01,643.805592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11891,2.181470,2.856510,2.104567,-2.178407,-1.703878,3.695532,1.550816,-0.578703,-1.348146,-0.389886,...,V1,V7,V1,V1,V4,V7,0,1,0.02,610.691598
11892,4.377081,5.994228,0.163564,3.400468,-2.569588,2.449087,-4.938499,-4.678133,1.490648,1.090210,...,V5,V20,V1,V1,V1,V7,0,1,0.04,625.181758
11893,2.498474,3.017701,1.886614,-2.600635,1.166471,4.855935,-2.661667,-1.057787,1.953560,0.725679,...,V8,V20,V1,V1,V3,V7,0,1,0.05,734.763451
11894,1.550833,3.418077,2.831779,-3.353625,-0.418480,3.588330,-2.657229,-1.607616,2.627191,0.580555,...,V5,V20,V1,V1,V1,V7,0,1,0.02,695.089436


## SEGMENTACION DE CLIENTES

In [12]:
# Reglas simples de segmentación
def segment_user(row):
    if row["churn_prob"] > 0.60:
        return "At-Risk"
    elif row["CLTV"] > data["CLTV"].quantile(0.75):
        return "High-Value"
    else:
        return "Safe"

data["segment"] = data.apply(segment_user, axis=1)

# Ver resumen
print(data["segment"].value_counts())

## Se generó un valor de CLTV basado en la probabilidad de churn y aleatoriedad controlada.

segment
Safe          8655
High-Value    2974
At-Risk        267
Name: count, dtype: int64


## custom messages using the output of your models.


In [ ]:
from openai import AzureOpenAI
client = AzureOpenAI(

api_key="",
 api_version="2023-05-15",
 azure_endpoint=""
)


# Redondear la probabilidad a porcentaje
probabilidad_churn_pct = round(data['churn_prob'] * 100, 2)

# Recorrer cada cliente
for i, probabilidad_churn in enumerate(data['churn_prob'].iloc[48:50], start=49):  #tomamos 2 clientes para ejemplo , 49 Y 59
    probabilidad_churn_pct = round(probabilidad_churn * 100, 2)

    # Clasificar el riesgo y crear prompt
    if data['segment'][i-1]=='At-Risk':
        nivel = "alto"
        prompt_usuario = (
            f"Cliente {i} tiene una probabilidad ALTA de abandono del {probabilidad_churn_pct}%. "
            "Redacta un mensaje corto para retenerlo, usando un tono empático y ofreciendo incentivos si es necesario."
        )
    elif data['segment'][i-1]=='High-Value':
        prompt_usuario = (
            f"Cliente {i} es un cliente de alto valor y tiene una probabilidad de abandono del {probabilidad_churn_pct}%. "
            "Escribe un mensaje corto que refuerce su compromiso y le recuerde los beneficios del servicio."
        )
    else:
        nivel = "No alto"
        prompt_usuario = (
            f"Cliente {i} no es un cliente de alto valor y tiene una probabilidad de abandono del {probabilidad_churn_pct}%. "
            "Escribe un mensaje corto y positivo en español para agradecerle su fidelidad."
        )

    # Llamar a Azure OpenAI
    respuesta = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Eres un experto en retención de clientes."},
            {"role": "user", "content": prompt_usuario}
        ]
    )

    mensaje_personalizado = respuesta.choices[0].message.content
    print(f"\n🧑 Cliente {i} — Riesgo: {nivel.upper()} ({probabilidad_churn_pct}%)")
    print("📩 Mensaje sugerido:")
    print(mensaje_personalizado)


🧑 Cliente 49 — Riesgo: NO ALTO (0.0%)
📩 Mensaje sugerido:
¡Gracias por confiar en nosotros! Valoramos muchísimo tu lealtad y estamos aquí para seguir ofreciéndote el mejor servicio. 😊

🧑 Cliente 50 — Riesgo: ALTO (74.0%)
📩 Mensaje sugerido:
Claro, aquí tienes un ejemplo de mensaje diseñado para retener al Cliente 50:  

---

**Asunto:** Queremos seguir siendo parte de tu vida  

Hola, [Nombre del Cliente],  

Sabemos lo importante que es para ti encontrar lo que realmente necesitas, y estamos aquí para asegurarnos de que tengas la mejor experiencia posible con nosotros. 💛  

Por eso, queremos agradecerte por formar parte de nuestra comunidad y ofrecerte un [incentivo: descuento, beneficio exclusivo o regalo, según tu negocio], como nuestra forma de decirte que valoramos mucho que sigas con nosotros.  

¿Hay algo más en lo que podamos ayudarte? Tu satisfacción es nuestra prioridad, y estamos aquí para escucharte.  

Haz clic aquí para canjear tu beneficio 👉 [enlace]  

Gracias por conf